# Convolutional Neural Network (CNN) Demonstration

## Import packages

In [155]:
! pip install torch torchvision pandas tqdm pathlib

In [156]:
import torch
import torch.nn as nn

import torchvision as tv
from torchvision import datasets
from torch.utils.data import DataLoader

import pandas as pd
from tqdm import tqdm
from pathlib import Path
from functools import reduce

## Models Implementation

### Feed-Forward Model Implementation

In [157]:
class FeedForwardClassifier(nn.Module):
  def __init__(self, input_shape, n_classes, hidden_units=1024, dropout_rate=0.25):
    super(FeedForwardClassifier, self).__init__()

    input_dim = reduce(lambda x, y: x * y, input_shape)

    self.classifier = nn.Sequential(
      nn.Flatten(),

      nn.Linear(input_dim, hidden_units),
      nn.ReLU(),
      nn.Dropout(dropout_rate),

      nn.Linear(hidden_units, hidden_units),
      nn.ReLU(),
      nn.Dropout(dropout_rate),

      nn.Linear(hidden_units, n_classes),
    )

  def forward(self, x):
    return self.classifier(x)

### CNN Model Implementation

In [158]:
class CNNClassifier(nn.Module):
  def __init__(self, input_shape, n_classes, cnn_hidden_units=32, linear_hidden_units=1024, dropout_rate=0.25):
    super(CNNClassifier, self).__init__()

    self.conv_block1 = nn.Sequential(
      nn.Conv2d(input_shape[0], cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),

      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),

      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Dropout(dropout_rate),
    )
    self.conv_block2 = nn.Sequential(
      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),

      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),

      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(dropout_rate),
    )
    linear_input_dim = cnn_hidden_units * (input_shape[1] // 4) * (input_shape[2] // 4)
    self.classifier = nn.Sequential(
      nn.Flatten(),

      nn.Linear(linear_input_dim, linear_hidden_units),
      nn.ReLU(),
      nn.Dropout(dropout_rate),

      nn.Linear(linear_hidden_units, n_classes),
    )

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)

    x = x.view(x.size(0), -1)

    x = self.classifier(x)

    return x

## Perform Experiment

### Define Accuracy Function

In [159]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct / len(y_pred)) * 100
  return acc

### Setup PyTorch device

In [160]:
device = (
  'cuda'
  if torch.cuda.is_available()
  else 'mps'
  if torch.backends.mps.is_available()
  else 'cpu'
)

### Define Function for Model Evaluation

In [161]:
def eval_model(model, data_loader, loss_fn, accuracy_fn):
  loss, acc = 0, 0

  model.eval()

  with torch.inference_mode():
    for x, y in data_loader:
      x, y = x.to(device), y.to(device)

      y_pred = model(x)

      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {'name': model.__class__.__name__, 'loss': loss.item(), 'acc': acc}

### Define Function for Model Evaluation

In [162]:
def get_n_params(model):
  total_params = 0

  for param in list(model.parameters()):
    num_elements = 1

    for size in list(param.size()):
      num_elements *= size

    total_params += num_elements

  return total_params

### Define Experiment Grid

In [163]:
class BalancedEMNIST(datasets.EMNIST):
  def __init__(self, *args, **kwargs):
    kwargs['split'] = 'balanced'
    super().__init__(*args, **kwargs)

In [164]:
example_datasets = (BalancedEMNIST, datasets.FashionMNIST, datasets.CIFAR100)
models = (FeedForwardClassifier, CNNClassifier)

### Perform Training and Evaluation

In [165]:
data_dir = Path('data')

batch_size = 32
epochs = 100

torch.manual_seed(42)

results_df = pd.DataFrame()

for Dataset in tqdm(example_datasets, desc='Datasets'):
  train_data = Dataset(
    root=data_dir / Dataset.__name__,
    train=True,
    download=True,
    transform=tv.transforms.ToTensor(),
    target_transform=None,
  )
  test_data = Dataset(
    root=data_dir / Dataset.__name__,
    train=False,
    download=True,
    transform=tv.transforms.ToTensor(),
  )

  image, label = train_data[0]
  input_shape = image.shape
  class_names = train_data.classes

  train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
  )
  test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False,
  )

  for Model in tqdm(models, desc='Models'):
    model = Model(input_shape=image.shape, n_classes=len(class_names)).to(device)
    n_params = get_n_params(model)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

    for epoch in tqdm(range(epochs), desc='Epochs'):
      train_loss = 0

      for batch, (x, y) in enumerate(train_dataloader):
        x, y = x.to(device), y.to(device)

        model.train()

        y_pred = model(x)

        loss = loss_fn(y_pred, y)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      train_loss /= len(train_dataloader)

      test_loss, test_acc = 0, 0

      model.eval()

      with torch.inference_mode():
        for x, y in test_dataloader:
          x, y = x.to(device), y.to(device)

          test_pred = model(x)

          test_loss += loss_fn(test_pred, y)

          test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

          test_loss /= len(test_dataloader)
          test_acc /= len(test_dataloader)

    results = eval_model(model, test_dataloader, loss_fn, accuracy_fn)
    row = pd.DataFrame({
      'dataset': Dataset.__name__,
      'model': Model.__name__,
      'n_params': n_params,
      'loss': results['loss'],
      'acc': results['acc'],
    }, index=[0])
    results_df = pd.concat([results_df, row], ignore_index=True)

results_df

Datasets:   0%|          | 0/3 [00:00<?, ?it/s]




































































































Epochs: 100%|██████████| 100/100 [19:43<00:00, 11.83s/it]


torch.Size([1, 28, 28])







































































































Datasets:  33%|███▎      | 1/3 [49:03<1:38:07, 2943.91s/it]




































































































Epochs: 100%|██████████| 100/100 [10:17<00:00,  6.18s/it]


torch.Size([1, 28, 28])







































































































Datasets:  67%|██████▋   | 2/3 [1:16:17<36:12, 2172.86s/it]

Files already downloaded and verified
Files already downloaded and verified







































































































Epochs: 100%|██████████| 100/100 [10:56<00:00,  6.57s/it]


torch.Size([3, 32, 32])







































































































Datasets: 100%|██████████| 3/3 [1:44:16<00:00, 2085.66s/it]


,dataset,model,n_params,loss,acc
0,BalancedEMNIST,FeedForwardClassifier,1901615,0.710763,85.831207
1,BalancedEMNIST,CNNClassifier,1682895,0.529978,88.249362
2,FashionMNIST,FeedForwardClassifier,1863690,0.426498,90.455272
3,FashionMNIST,CNNClassifier,1644970,0.299659,92.501997
4,CIFAR100,FeedForwardClassifier,4298852,3.634229,21.605431
5,CIFAR100,CNNClassifier,2229316,3.554307,28.464457
